# Import packages

In [13]:
# !pip install -U h5coro
# !pip show h5coro # should be 0.0.7

In [14]:
from datetime import datetime, timezone, timedelta
import earthaccess

In [17]:
# import the local icepyx package for testing the new module
import sys
import os

# Add the project directory to sys.path
local_package_path = os.path.abspath(os.path.join('..', '..', '..'))
sys.path.insert(0, local_package_path)
from icepyx.core import hdf5_arrow_table

# Generate credentials

In [18]:
earthaccess.login()

In [19]:
nsidc_aws_creds = earthaccess.get_s3_credentials(daac='NSIDC')
nsidc_credentials = dict(
    aws_access_key_id=nsidc_aws_creds['accessKeyId'],
    aws_secret_access_key=nsidc_aws_creds['secretAccessKey'],
    aws_session_token=nsidc_aws_creds['sessionToken']
)
lpdaac_aws_creds = earthaccess.get_s3_credentials(daac='LPDAAC')
lpdaac_credentials = dict(
    aws_access_key_id=lpdaac_aws_creds['accessKeyId'],
    aws_secret_access_key=lpdaac_aws_creds['secretAccessKey'],
    aws_session_token=lpdaac_aws_creds['sessionToken']
)

# Query for data uris (S3 URLs)

In [7]:
start = datetime(2021, 11, 1, tzinfo=timezone.utc)
end = start + timedelta(days=7)

atl08_results = earthaccess.search_data(
    short_name="ATL08",
    cloud_hosted=True,
    temporal=(start, end),
    bounding_box=(-90,-56,-32,14),
    count=-1
)
atl08_links = [result.data_links(access="direct")[0] for result in atl08_results]

In [8]:
gedil2b_results = earthaccess.search_data(
    short_name="GEDI02_B",
    cloud_hosted=True,
    temporal=(start, end),
    bounding_box=(-90,-56,-32,14),
    count=-1
)
gedil2b_links = [result.data_links(access="direct")[0] for result in gedil2b_results]

# Construct dataset arguments

In [9]:
atl08_variables = {
    "variables": {
        "h_canopy": {"dataset": "/gt1l/land_segments/canopy/h_canopy" },
        "dem_h": {"dataset": "/gt1l/land_segments/dem_h" }
    },
    "latitude": {"dataset": "/gt1l/land_segments/latitude" },
    "longitude": {"dataset": "/gt1l/land_segments/longitude" }
}

gedil2b_variables = {
    "variables": {
        "cover": {"dataset": "BEAM0000/cover" }
    },
    "latitude": {"dataset": "/BEAM0000/geolocation/lat_highestreturn" },
    "longitude": {"dataset": "/BEAM0000/geolocation/lon_highestreturn" }
}

# Create table for 1 file

Just to make sure it works.

In [20]:
creator = hdf5_arrow_table.HDF5ArrowTableCreator(
    uri=atl08_links[0],
    datasets=atl08_variables,
    credentials=nsidc_credentials,
    mask_using='h_canopy'
)
table = creator.create_table()

1724362594.803694 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values


# Create table for all files

In [21]:
%%time
gedil2b_table = hdf5_arrow_table.concat_tables(
    uris=gedil2b_links,
    datasets=gedil2b_variables,
    credentials=lpdaac_credentials,
    mask_using='cover'
)

only handle 1-dimensional arrays
only handle 1-dimensional arrays
CPU times: user 97.7 ms, sys: 101 ms, total: 199 ms
Wall time: 47.1 s


In [22]:
atl08_table = hdf5_arrow_table.concat_tables(
    uris=atl08_links,
    datasets=atl08_variables,
    credentials=nsidc_credentials,
    mask_using='h_canopy'
)

1724362649.705434 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362649.756840 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362649.788502 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362649.849726 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362650.837690 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362650.869672 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1724362651.189878 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/

only handle 1-dimensional arrays


# Write to and read from geoparquet, load into pandas data frame

(Geo)parquet is a file format, so can be used to store pyarrow tables. Another popular in-memory format is (geo)pandas dataframes. They all work together very well!

In [28]:
import pyarrow.parquet as pq

parquet_file = 'gedil2b.parquet'

# Write pyarrow table to parquet
pq.write_table(gedil2b_table, parquet_file)

# Read parquet file into pyarrow table
pyarrow_table = pq.read_table(parquet_file, memory_map=True)

# Create pandas data from from pyarrow table
pyarrow_table.to_pandas()

,cover,geometry
0,0.406484,"[-165.78689544834205, 51.76469622868379]"
1,0.068555,"[-164.87850504271515, 51.731664667530836]"
2,0.143258,"[-164.87685248871196, 51.731597125786]"
3,0.174157,"[-164.74680739065076, 51.72608301395156]"
4,0.364871,"[-164.72461010879564, 51.72511593042972]"
...,...,...
2792206,0.014901,"[-29.673782908470677, -0.2996122838878096]"
2792207,0.073731,"[-29.671706888931052, -0.2966675989950678]"
2792208,0.086019,"[-29.670223296663064, -0.29456361315940705]"
2792209,0.079618,"[-29.66725711371029, -0.2903559417091873]"


In [29]:
# You can also read directly into a pandas data frame
import pandas as pd
pd.read_parquet(parquet_file, engine='pyarrow')

,cover,geometry
0,0.406484,"[-165.78689544834205, 51.76469622868379]"
1,0.068555,"[-164.87850504271515, 51.731664667530836]"
2,0.143258,"[-164.87685248871196, 51.731597125786]"
3,0.174157,"[-164.74680739065076, 51.72608301395156]"
4,0.364871,"[-164.72461010879564, 51.72511593042972]"
...,...,...
2792206,0.014901,"[-29.673782908470677, -0.2996122838878096]"
2792207,0.073731,"[-29.671706888931052, -0.2966675989950678]"
2792208,0.086019,"[-29.670223296663064, -0.29456361315940705]"
2792209,0.079618,"[-29.66725711371029, -0.2903559417091873]"


# Visualize with lonboard

In [30]:
# how many points?
gedil2b_table['cover'].length() + atl08_table['h_canopy'].length()

3029759

In [31]:
import lonboard

atl08_layer = lonboard.ScatterplotLayer(table=atl08_table)
gedil2b_layer = lonboard.ScatterplotLayer(table=gedil2b_table)

In [32]:
m = lonboard.Map([atl08_layer, gedil2b_layer])
m

Map(layers=[ScatterplotLayer(table=pyarrow.Table
dem_h: float
h_canopy: float
geometry: fixed_size_list<item: …

# Todos:

* resolve CRS error
* see if DIMENSION_LIST error is resolveable